In [73]:
import numpy as np
from anytree import RenderTree

In [1]:
%load_ext autoreload
%autoreload 2

from tictac_rl import TicTacToe, build_minmax_tree, best_move

In [2]:
env = TicTacToe(3, 3, 3)

In [3]:
tree, hash_table = build_minmax_tree(env)

In [18]:
env.reset()

In [19]:
(hash_state, *_), *_  = env.step((0, 0))
is_max = False

In [20]:
env.printBoard()

-------------
| x |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------


In [21]:
while True:
   is_max = not is_max
   node = hash_table[hash_state]
   move = best_move(node, is_max)
   print(move)
   (hash_state, *_), reward, is_end  = env.step(move)
   env.printBoard()
   print("-" * 50)
   if is_end:
       break

(1, 1)
-------------
| x |   |   | 
-------------
|   | o |   | 
-------------
|   |   |   | 
-------------
--------------------------------------------------
(0, 1)
-------------
| x | x |   | 
-------------
|   | o |   | 
-------------
|   |   |   | 
-------------
--------------------------------------------------
(0, 2)
-------------
| x | x | o | 
-------------
|   | o |   | 
-------------
|   |   |   | 
-------------
--------------------------------------------------
(2, 0)
-------------
| x | x | o | 
-------------
|   | o |   | 
-------------
| x |   |   | 
-------------
--------------------------------------------------
(1, 0)
-------------
| x | x | o | 
-------------
| o | o |   | 
-------------
| x |   |   | 
-------------
--------------------------------------------------
(1, 2)
-------------
| x | x | o | 
-------------
| o | o | x | 
-------------
| x |   |   | 
-------------
--------------------------------------------------
(2, 1)
-------------
| x | x | o | 
----------

In [9]:
env.step((0, 2))

(('200021022', array([[1, 2]], dtype=int64), 1), 0, False)